In [2]:
%run ./read_file.ipynb 

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-261c36bd-34a2-4ca5-b557-57d9eea8c035;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 270ms :: artifacts dl 6ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/08 03:06:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 03:06:15 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/08 03:06:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_labels = readCheckpoint_bcm('labels')
#create a timestamp column and drop the old timestamp - does not appear necessary
df_labels = df_labels.withColumn('ts2', to_timestamp(col('timestamp'))).drop('timestamp')
#check new col data types
df_labels.printSchema()

 3:06AM UTC on Mar 08, 2023 --- read and cache time: 1.811981201171875e-05 seconds ---
root
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- ts2: timestamp (nullable = true)



In [38]:
def build_adjaceny(df, dataset = 'ecar'):
    
    start_time = time.time()
    
    if dataset == 'labels':
        df = df.sort(col("ts2").asc())
    else : 
        #first step sort the data and check
        df = df.sort(col("timestamp").asc())
    
    #now get overlapping connections

    #build collections of each type of ID
    #TODO make this one select statement
    col_objID = df.select("objectID").distinct().collect()
    col_actID = df.select("actorID").distinct().collect()
    
    print(len(col_actID))

    #put these columns into a dictionary and turn into a spark data frame
    data = [{"actorIDs" : [row.actorID for row in col_actID], \
                   "objectIDs" : [row.objectID for row in col_objID]}]

    id_intersect = spark.createDataFrame(data)

    #compare ID pools to find actIDs that are nodes
    id_intersect = id_intersect.withColumn("id_int", \
                                           array_intersect(col("actorIDs"),col("objectIDs")))

    #now build a list of the nodal actIDs and print the length
    actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]
    
    print("length actIDs: "+str(len(actIDs)))
    print("elapsed time: "+ str(time.time() - start_time))
    #create ref dict
    dict_node_edge_events = {}
    count = 0 
    
    #for all actors in actIDs
    for aID in actIDs: 
        
        birth_event = df.where((col("objectID") == aID) \
                               & (col("actorID")!= aID)).limit(1).collect()
        
        if len(birth_event) > 0: 
            first_id = birth_event[0]['id']

            #now add this to the dictionary as a key
            first_event = str(aID+':'+first_id)
            dict_node_edge_events[first_event] = []

            #now add all the events where the actorID is present but not the objectID, group by unique object ID
            #and pull out the first event - and I checked and believe the sorting holds through grouping
            col_select_children_events = df.select('objectID','id')\
            .where((col("actorID") == aID) & \
                     (col("objectID") != aID)).groupBy("objectID").agg(first("id")).collect() 

            #add children objects into dictioinary 
            dict_node_edge_events[first_event] = [str(row["objectID"]+':'+row["first(id)"]) for row in col_select_children_events]
        else:
            count += 1
    
    print("no birth event for " + count+" events.")
    print("elapsed time 2: "+ str(time.time() - start_time))
    return dict_node_edge_events

In [5]:
df_eps = readCheckpoint()

 3:19AM UTC on Mar 08, 2023 --- read and cache time: 0.6242907047271729 seconds ---


In [39]:
start_time = time.time()
test_dict = build_adjaceny(df_eps)
print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- elapsed: " % (time.time() - start_time))

101841


length actIDs: 81891
elapsed time: 27.920552730560303


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 8) / 9]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [31]:
samp = df_eps.where((col("objectID") == '661181c7-4f8e-4327-9279-8857e6d79f8b') \
                               & (col("actorID")!= '661181c7-4f8e-4327-9279-8857e6d79f8b')).limit(1).collect()
print(len(samp))

0


In [32]:
df_eps.where((col("objectID") == '661181c7-4f8e-4327-9279-8857e6d79f8b') \
                               & (col("actorID")!= '661181c7-4f8e-4327-9279-8857e6d79f8b')).toPandas()

,id,timestamp,objectID,actorID,object,action,hostname,user_name,privileges,image_path,parent_image_path,new_path,file_path,direction,logon_id,requesting_domain,requesting_user,event_minute,event_day,event_hour


In [28]:
samp[0]['id']

'73f79c2a-288b-49ee-9abe-a33a943ee45f'